## Create and save a scaler

In [39]:
# --> Save the scaler object to be used in the prediction

import pandas as pd
from sklearn.preprocessing import StandardScaler


df = pd.read_csv('../dataset/Mosaico/train.csv')
numeric_features = ['bedrooms', 'bathrooms', 'area', 'zipcode']

scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])

df.head()

import pickle

with open('../models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


## Preprocess images

In [40]:
from PIL import Image
from torchvision import transforms
import torch

def preprocess_image(bedroom_path: str, bathroom_path: str, kitchen_path: str, frontal_path: str) -> torch.Tensor:
    """
    This function receives the paths to the images and returns a tensor that can be used as input to the model.

    Parameters:
        - bedroom_path: str, path to the bedroom image.
        - bathroom_path: str, path to the bathroom image.
        - kitchen_path: str, path to the kitchen image.
        - frontal_path: str, path to the frontal image.

    Returns:
        - torch.Tensor, tensor that can be used as input to the model.
    """
    bathroom_image = Image.open(bathroom_path)
    bedroom_image = Image.open(bedroom_path)
    kitchen_image = Image.open(kitchen_path)
    frontal_image = Image.open(frontal_path)

    # Resize imagenes
    bathroom_image = bathroom_image.resize((200, 200))
    bedroom_image = bedroom_image.resize((200, 200))
    kitchen_image = kitchen_image.resize((200, 200))
    frontal_image = frontal_image.resize((200, 200))

    # Create mosaico
    mosaic = Image.new('RGB', (400, 400))
    mosaic.paste(bathroom_image, (0, 0))
    mosaic.paste(bedroom_image, (200, 0))
    mosaic.paste(kitchen_image, (0, 200))
    mosaic.paste(frontal_image, (200, 200))
    
    # Transformar imagen
    transform = transforms.Compose([
        transforms.Resize(256),                               # Redimensionar a 256x256
        transforms.CenterCrop(224),                           # Recortar al centro para obtener 224x224
        transforms.ToTensor(),                                # Convertir la imagen a un tensor
        transforms.Normalize(
            (0.485, 0.456, 0.406),                            # Media de ImageNet
            (0.229, 0.224, 0.225)                             # Desviación estándar de ImageNet
        )
    ])

    return transform(mosaic).unsqueeze(0)

In [47]:
# path to images
bathroom_path = '../dataset/Houses-dataset/Houses Dataset/301_bathroom.jpg'
kitchen_path = '../dataset/Houses-dataset/Houses Dataset/301_kitchen.jpg'
frontal_path = '../dataset/Houses-dataset/Houses Dataset/301_frontal.jpg'
bedroom_path = '../dataset/Houses-dataset/Houses Dataset/301_bedroom.jpg'

# preprocess image
image_tensor = preprocess_image(bedroom_path, bathroom_path, kitchen_path, frontal_path)

## Preprocess numeric data

In [42]:
import pickle
import numpy as np


def preprocess_numeric_features(features: list, scaler_path: str) -> torch.Tensor:
    """
    This function receives the numeric features and the path to the 
    scaler object and returns a tensor that can be used as input to the model.

    Parameters:
        - features: list, list with the numeric features.
        - scaler_path: str, path to the scaler object.

    Returns:
        - torch.Tensor, tensor that can be used as input to the model.
    """

    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)

    features = scaler.transform([features])

    return torch.tensor(features, dtype=torch.float32)

In [48]:
numeric_features = [5,5.0,4014,92880]
scaler_path = '../models/scaler.pkl'

numeric_tensor = preprocess_numeric_features(numeric_features, scaler_path)

c:\Users\uriel\OneDrive\Escritorio\Uriel\IPN\ESCOM\TT\Codigo\my_code\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## Predict the price of a house

In [44]:
from torchvision import models
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        # Cargar el modelo preentrenado ResNet50
        self.image_features_ = models.resnet50(pretrained=True)

        # Eliminar la última capa de clasificación (fully connected) de ResNet50
        self.image_features_ = nn.Sequential(*list(self.image_features_.children())[:-1])

        # Procesamiento de las características numéricas (4 características)
        self.numeric_features_ = nn.Sequential(
            nn.Linear(4, 64),  # Aquí usas 4 datos numéricos
            nn.GELU(),  # GELU en lugar de ReLU
            nn.Dropout(),
            nn.Linear(64, 64*3),
            nn.GELU(),  # GELU
            nn.Dropout(),
            nn.Linear(64*3, 64*3*3),
            nn.GELU(),  # GELU
        )

        # Capa final que combina las características visuales y numéricas
        self.combined_features_ = nn.Sequential(
            nn.Linear(2048 + 64*3*3, 64*3*3*2*2),  # 2048 provienen de ResNet50 + numéricas
            nn.GELU(),  # GELU
            nn.Dropout(),
            nn.Linear(64*3*3*2*2, 64*3*3*2),
            nn.GELU(),  # GELU
            nn.Linear(64*3*3*2, 64),
            nn.Linear(64, 1),  # Predicción final
        )

    def forward(self, x, y):
        # Pasar las imágenes por ResNet50 para obtener las características visuales
        x = self.image_features_(x)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        # print(x.shape)

        # Pasar las características numéricas por la red densa
        y = self.numeric_features_(y)

        # Combinar características visuales y numéricas
        z = torch.cat((x, y), dim=1)
        # print(z.shape)

        # Pasar las características combinadas por las capas finales
        z = self.combined_features_(z)

        return z.squeeze(1)

In [49]:
import torch

# Define the paths
ruta_modelo = '../models/'
nombre_modelo = 'model_resnet50_l2_lambda9.pth'

# Load model
model = torch.load(f"{ruta_modelo}{nombre_modelo}", map_location=torch.device('cpu'))
model.eval()

NeuralNetwork(
  (image_features_): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0

In [50]:
# Make prediction
with torch.no_grad():
    prediction = model(image_tensor, numeric_tensor).item()

print(f"Predicted house price: ${prediction:.2f}")

Predicted house price: $639500.25
